In [4]:
!pip install torch torchvision

In [13]:
import torch, torchvision
from torch import nn,optim
from torch.autograd import Variable as var 

In [6]:
n_batch = 64
learning_rate = 0.01
n_epoch = 3
n_print = 10

In [7]:
#### Part I : Loading Your Data 

T = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.FashionMNIST('fashion-mnist_data',train=True,download=True,transform=T)
val_data = torchvision.datasets.FashionMNIST('fashion-mnist_data',train=False,download=True,transform=T)

train_dl = torch.utils.data.DataLoader(train_data,batch_size = n_batch)
val_dl = torch.utils.data.DataLoader(val_data,batch_size = n_batch)

Extracting fashion-mnist_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to fashion-mnist_data/FashionMNIST/raw


Extracting fashion-mnist_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to fashion-mnist_data/FashionMNIST/raw


Extracting fashion-mnist_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to fashion-mnist_data/FashionMNIST/raw




Extracting fashion-mnist_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to fashion-mnist_data/FashionMNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [8]:
#### Part II : Writing the Network
class myCNN(nn.Module):
  def __init__(self):
    super(myCNN,self).__init__()
    self.cnn1 = nn.Conv2d(1,3,3)
    self.cnn2 = nn.Conv2d(3,2,5)
    self.linear = nn.Linear(968,10)
    self.relu = nn.ReLU()
  
  def forward(self,x):
    n = x.size(0)
    x = self.relu(self.cnn1(x))
    x = self.relu(self.cnn2(x))
    x = x.view(n,-1)
    x = self.linear(x)
    return x

In [ ]:
 #Writing the main Training loop

mycnn = myCNN().cuda()
cec = nn.CrossEntropyLoss()
optimizer = optim.Adam(mycnn.parameters(),lr = learning_rate)

def validate(model,data):
  # To get validation accuracy = (correct/total)*100.
  total = 0
  correct = 0
  for i,(images,labels) in enumerate(data):
    images = var(images.cuda())
    x = model(images)
    value,pred = torch.max(x,1)
    pred = pred.data.cpu()
    total += x.size(0)
    correct += torch.sum(pred == labels)
  return correct*100./total

for e in range(n_epoch):
  for i,(images,labels) in enumerate(train_dl):
    images = var(images.cuda())
    labels = var(labels.cuda())
    optimizer.zero_grad()
    pred = mycnn(images)
    loss = cec(pred,labels)
    loss.backward()
    optimizer.step()
    if (i+1) % n_print == 0:
      accuracy = float(validate(mycnn,val_dl))
      print('Epoch :',e+1,'Batch :',i+1,'Loss :',float(loss.data),'Accuracy :',accuracy,'%')

In [16]:
#Test model
for i,(images,labels) in enumerate(val_dl):
    images = var(images.cuda())
    labels = var(labels.cuda())
    optimizer.zero_grad()
    pred = mycnn(images)
    loss = cec(pred,labels)
    loss.backward()
    optimizer.step()
    if (i+1) % n_print == 0:
      accuracy = float(validate(mycnn,val_dl))
      print('Epoch :',e+1,'Batch :',i+1,'Loss :',float(loss.data),'Accuracy :',accuracy,'%')

Epoch : 3 Batch : 10 Loss : 0.4006693959236145 Accuracy : 88.5199966430664 %
Epoch : 3 Batch : 20 Loss : 0.3062759041786194 Accuracy : 88.45999908447266 %
Epoch : 3 Batch : 30 Loss : 0.15869677066802979 Accuracy : 87.63999938964844 %
Epoch : 3 Batch : 40 Loss : 0.26217541098594666 Accuracy : 88.44000244140625 %
Epoch : 3 Batch : 50 Loss : 0.4559166729450226 Accuracy : 87.7699966430664 %
Epoch : 3 Batch : 60 Loss : 0.4874069392681122 Accuracy : 87.93000030517578 %
Epoch : 3 Batch : 70 Loss : 0.17820703983306885 Accuracy : 88.55000305175781 %
Epoch : 3 Batch : 80 Loss : 0.37952765822410583 Accuracy : 88.44999694824219 %
Epoch : 3 Batch : 90 Loss : 0.4089038372039795 Accuracy : 89.33000183105469 %
Epoch : 3 Batch : 100 Loss : 0.23234571516513824 Accuracy : 89.12000274658203 %
Epoch : 3 Batch : 110 Loss : 0.3387977182865143 Accuracy : 88.0999984741211 %
Epoch : 3 Batch : 120 Loss : 0.16905738413333893 Accuracy : 88.66000366210938 %
Epoch : 3 Batch : 130 Loss : 0.38611742854118347 Accuracy 